This Experiement was made with textblob (https://textblob.readthedocs.io/en/dev/), a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.
At first we kept it simple. We trained it with an array of sentences. We trained the classifier, what is a spam and what ham. 
Most of the time, the classifier was correct.


In [1]:
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
from textblob.classifiers import DecisionTreeClassifier


#Custom training data with NaiveBayes
train = [
    ('Buy cheap drugs', 'spam'),
    ('Cheap viagra', 'spam'),
    ('Win 1000 dollar', 'spam'),
    ('Greatings, how are you?', 'ham'),
    ('What an awesome picture', 'ham'),
    ('Send me your adress', 'ham'),
    ('viagra', 'spam')
]
test = [
    ('Cheap drugs', 'spam'),
    ('Buy viagra', 'spam'),
    ("Send me your picture", 'ham'),
    ("Provide you CV", 'ham')
]

cl = NaiveBayesClassifier(train)

# Classify some text
print(cl.classify("Buy drugs"))  # "spam"
print(cl.classify("How are you?"))   # "ham"

# Classify a TextBlob
blob = TextBlob("Buy cheap drugs for only 100 dollar. Send me your adress and you will get viagra for free!", classifier=cl)
print(blob)
print(blob.classify())

for sentence in blob.sentences:
    print(sentence)
    print(sentence.classify())

# Compute accuracy
print("Accuracy: {0}".format(cl.accuracy(test)))

# Show 5 most informative features
cl.show_informative_features()

spam
ham
Buy cheap drugs for only 100 dollar. Send me your adress and you will get viagra for free!
spam
Buy cheap drugs for only 100 dollar.
spam
Send me your adress and you will get viagra for free!
ham
Accuracy: 0.75
Most Informative Features
        contains(viagra) = False             ham : spam   =      1.8 : 1.0
          contains(Send) = False            spam : ham    =      1.4 : 1.0
           contains(how) = False            spam : ham    =      1.4 : 1.0
          contains(What) = False            spam : ham    =      1.4 : 1.0
       contains(awesome) = False            spam : ham    =      1.4 : 1.0
     contains(Greatings) = False            spam : ham    =      1.4 : 1.0
        contains(adress) = False            spam : ham    =      1.4 : 1.0
       contains(picture) = False            spam : ham    =      1.4 : 1.0
          contains(your) = False            spam : ham    =      1.4 : 1.0
            contains(an) = False            spam : ham    =      1.4 : 1.0


Now we decided to do it with the html files from standard. We wanted to classify only the header, so the classifier should say if the text is a header or not. Therefore we extracted the text from the html file and put all header in an array which was used as the training data. This task was very difficult, because of special characters and the way how the data must be modified to be readable for the classifier. So we extracted some header by our own and put it in an array. With this we trained the classifier, but we have not considered, that textblob read every sentence as its own till a dot. Therefore every sentence was an header for textblob, that’s why we considered a natural language processor as not suitable for our task/goal.


In [2]:
from bs4 import BeautifulSoup
import os, sys
import urllib
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier


#Hand extracted headers from articles from standard.at
train = [
        ('1.000 Folgen: Das Quiz zum Tatort-Jubiläum', 'header'),
        ('1.000 Tatorte in Berlin mit Schimanskis blutbefleckter Jacke', 'header'),
        ('1. April: Photobombs with David Hasselhoff und Hofer-Roboter', 'header'),
        ('Zehn Jahre Comic-Vertrieb Pictopia: Eine One-Man-Show', 'header'),
        ('Zehn-Minuten-Küche: Dem Fisch Dampf machen', 'header'),
        ('Zehn spektakuläre Pools mit Aussicht', 'header'),
        ('11.500 Wahlberechtigte mehr als im April und Mai', 'header'),
        ('13-Jähriger angeschossen: Prozess um versuchten Mord in Wien', 'header'),
        ('13 Tote durch Brand in Lederfabrik nahe Neu Delhi', 'header'),
        ('14 Kilometer langer Tunnel durch die Anden geplant', 'header'),
        ('14 Tipps für Hausstaubmilbenallergiker', 'header'),
        ('Gewaltvideo: Polizei nimmt Hauptverdächtige fest, neuer Fall in Wien', 'header'),
        ('3,2 Milliarden Jahre alte Lebensspuren in Südafrika entdeckt', 'header'),
        ('Schlaflabor: 3D-videoüberwacht im Bett', 'header'),
        ('5th Avenue teuerste Shoppingmeile der Welt, Kohlmarkt auf Platz zehn', 'header'),
        ('5th Avenue teuerste Shoppingmeile der Welt, Kohlmarkt Platz zehn', 'header'),
        ('Sechs Tipps für die gesunde Sauna', 'header'),
        ('Sieben Risikofaktoren für Alzheimer', 'header'),
        ('Acht Tipps für "Fit durch den Winter"', 'header'),
        ('Neu-Delhi – In der Nähe der indischen Hauptstadt Neu Delhi sind am Freitag durch einen Brand in einer Lederfabrik 13 Menschen ums Leben gekommen. Bei den Toten handelte es sich nach Polizeiangaben um Beschäftigte des Unternehmens, das sich in der östlichen Vorstadt Sahibabad befindet. Die Arbeiter hatten demnach in der Firma übernachtet und wurden von dem Brand im Schlaf überrascht. Zwei oder drei Arbeiter seien verletzt ins Krankenhaus gebracht worden, sagte Polizeisprecher Bhagwat Singh.', 'NOT'),
        ('In Indien gibt es immer wieder schwere Unfälle an Arbeitsstätten. Im Oktober kamen bei einer Explosion in einem Unternehmen für Feuerwerkskörper im Bundesstaat Tamil Nadu acht Arbeiter ums Leben. Bei einem ähnlichen Unglück gab es im Mai 2014 im Bundesstaat Madhya Pradesh 15 Tote. (APA, 11.11.2016)', 'NOT')

]

#train the classifier
cl = NaiveBayesClassifier(train)
#open a file with headers
file=open("test.txt");
t=file.read();
blob = TextBlob(t, classifier=cl)
print(blob)
print(blob.classify())
for sentence in blob.sentences:
    print(sentence)
    print(sentence.classify())

# Compute accuracy
print("Accuracy: {0}".format(cl.accuracy(test)))

# Show 5 most informative features
cl.show_informative_features()


13 Tote durch Brand in Lederfabrik nahe Neu Delhi

Feuerwehr: Unternehmen arbeitete wahrscheinlich illegal

Neu-Delhi – In der Nähe der indischen Hauptstadt Neu Delhi sind am Freitag durch einen Brand in einer Lederfabrik 13 Menschen ums Leben gekommen. Bei den Toten handelte es sich nach Polizeiangaben um Beschäftigte des Unternehmens, das sich in der östlichen Vorstadt Sahibabad befindet.

Die Arbeiter hatten demnach in der Firma übernachtet und wurden von dem Brand im Schlaf überrascht. Zwei oder drei Arbeiter seien verletzt ins Krankenhaus gebracht worden, sagte Polizeisprecher Bhagwat Singh.
Unternehmen wahrscheinlich illegal

Das Unternehmen arbeitete nach Einschätzung der Feuerwehr wahrscheinlich illegal. "Soweit wir sehen, war nichts in Ordnung", sagte der Feuerwehrmann Abbas Hussain. "Das Unternehmen war bestimmt nicht legal, wir können es aber erst nach einer korrekten Untersuchung bestätigen."

In Indien gibt es immer wieder schwere Unfälle an Arbeitsstätten. Im Oktober kame